# Subclassing 소개

## 기존 nn.Sequential

In [29]:
import torch
import torch.nn as nn
from debugpy.launcher import channel
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as  transforms

In [2]:
model = nn.Sequential()
model.append(module=nn.Linear(784, 256, bias=True))
model.append(module=nn.ReLU())
model.append(module=nn.Linear(256, 10, bias=True))

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

## Subclassing 활용

In [4]:
# Subclassing
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(784, 10, bias=True)
        self.linear2 = nn.Linear(256, 10, bias=True)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        return self.linear2(x)

In [ ]:
## Subclassing - Sequential 이용
class CustomModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(784, 10, bias=True),
            nn.ReLU(),
            nn.Linear(256, 10, bias=True),
        )


    def forwar(self, x):
        x = self.linear_stack(x)
        return x

In [6]:
import lightning.pytorch as pl

In [30]:
## LigtningModule 이용
class LMnist(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # MNIST 이미지의 크기 (1, 28, 28) -> (채널, width, height)
        self.linear1 = nn.Linear(28*28, 256)
        self.linear2 = nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, width, height = x.size()

        #Flatten
        x = x.view(batch_size, -1) # batch_size 1*28*28

        # Layer1
        x = self.linear1(x)
        x = torch.relu(x)

        # Layer2
        x = self.linear2(x)

        x = torch.log_softmax(x, dim=1)
        return x

    def cross_entropy_loss(self, logits, labels):
        # softmax + nll_loss
        return nn.functional.nll_loss(logits, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('val_loss', loss)

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('test_loss', loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

## 데이터 가공

In [31]:
from torch.utils.data import random_split

transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# 데이터 준비
mnist_train = MNIST("./", train=True, download=True, transform=transforms)
mnist_train, mnist_val = random_split(mnist_train, [0.9, 0.1])

mnist_test = MNIST("./", train=False, download=True, transform=transforms)

In [32]:
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=64)
test_loader = DataLoader(mnist_test, batch_size=64)

In [33]:
# 사용 가능한 GPU 개수 확인
gpu_count = torch.cuda.device_count()
print(f"사용 가능한 GPU 개수: {gpu_count}")

사용 가능한 GPU 개수: 1


In [34]:
model = LMnist()
trainer = pl.Trainer(default_root_dir="../lightning_logs", max_epochs=10, accelerator="gpu", devices=1)
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params | Mode 
-------------------------------------------
0 | linear1 | Linear | 200 K  | train
1 | linear2 | Linear | 2.6 K  | train
-------------------------------------------
203 K     Trainable params
0         Non-trainable params
203 K     Total params
0.814     Total estimated model params size (MB)
2         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [35]:
trainer.test(model, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\user\anaconda3\envs\namduhus_GPU\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.11870215088129044    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.11870215088129044}]